In [25]:
import pandas as pd
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import minmax_scale

from notears import linear, nonlinear, utils

In [26]:
def var_sort_lin(X_norm, d, sorting):
    X_varsort = X.copy()
    X_varsort[:, sorting] *= np.linspace(1, d, d)
    return X_varsort

def var_sort_log(X_norm, d, sorting):
    X_varsorted = X_norm.copy()

    vars = np.logspace(2, d+1, d, base=0.5)
    vars = np.full(vars.shape, vars.max()) - vars
    vars = minmax_scale(vars, feature_range=(1, d))

    print(vars)

    X_varsorted[:, sorting] *= vars
    return X_varsorted

In [17]:
def varsortability(X, W, tol=1e-9):
    """ Takes n x d data and a d x d adjaceny matrix,
    where the i,j-th entry corresponds to the edge weight for i->j,
    and returns a value indicating how well the variance order
    reflects the causal order. """
    E = W != 0
    Ek = E.copy()
    var = np.var(X, axis=0, keepdims=True)

    n_paths = 0
    n_correctly_ordered_paths = 0

    for _ in range(E.shape[0] - 1):
        n_paths += Ek.sum()
        n_correctly_ordered_paths += (Ek * var / var.T > 1 + tol).sum()
        n_correctly_ordered_paths += 1/2*(
            (Ek * var / var.T <= 1 + tol) *
            (Ek * var / var.T >  1 - tol)).sum()
        Ek = Ek.dot(E)

    return n_correctly_ordered_paths / n_paths

In [36]:
d = 10
n = 100
s0 = 10
graph_type = "ER" 

B_true = utils.simulate_dag(d, s0, graph_type)
W = utils.simulate_parameter(B_true)
X = utils.simulate_linear_sem(B_true, n, "gauss")

In [37]:
X.std(axis=0)

array([1.00368651, 2.85957702, 0.96485369, 3.66331643, 1.30488458,
       1.01703322, 1.43862061, 1.65835429, 1.27777767, 0.9692194 ])

In [38]:
g = ig.Graph.Adjacency(B_true, loops=False)
g.vs["label"] = list(range(d))

sorting = g.topological_sorting()

print(sorting)

[0, 2, 5, 9, 4, 8, 6, 7, 1, 3]


In [39]:
# ORIGINAL
print("VS-original", varsortability(X, W))

# NORMALIZE
X = (X  - X.mean(axis=0)) / X.std(axis=0)
print("VS-normalised", varsortability(X, W))


# CONTROL VARSORT
X = var_sort_log(X, d, sorting)
print("VS-controlled-lin", varsortability(X, W))

VS-original 1.0
VS-normalised 0.5
[ 1.          5.50880626  7.76320939  8.89041096  9.45401174  9.73581213
  9.87671233  9.94716243  9.98238748 10.        ]
VS-controlled-lin 1.0


In [40]:
X

array([[-4.51877946e-02,  1.93697794e+00,  1.08760621e+01,
         3.78040201e-01,  1.38335156e+01,  1.08238691e+01,
        -2.75365086e+00,  4.21238804e+00, -1.44681251e+00,
        -6.22798377e-01],
       [ 3.51164582e-01,  1.59499284e+01,  4.15183399e+00,
         2.35022574e+00,  7.89460835e-01,  1.53693265e+01,
         1.18517088e+01,  9.62057374e+00, -7.38579823e+00,
         1.18855578e+00],
       [-3.71442441e-01, -6.41405266e+00, -9.97299095e+00,
        -4.10051902e+00, -4.29194229e+00, -8.43677530e+00,
        -9.63446522e+00, -9.58762222e+00,  2.30941566e+00,
         4.49295566e+00],
       [-3.59091661e-01,  5.56357998e+00, -4.03008407e+00,
         3.67517229e+00,  1.13523246e+00,  1.32518425e+00,
         3.18975295e+00,  1.30853831e+00, -5.82079738e+00,
         5.24614477e+00],
       [ 1.45952975e-01,  2.67642751e+00, -3.14257192e+00,
         1.36414675e+00,  1.19238540e+01,  2.47870405e-01,
         5.65671072e+00,  1.36584745e+00, -7.45265688e+00,
         6.

In [41]:
X.std(axis=0)

array([ 1.        ,  9.98238748,  5.50880626, 10.        ,  9.45401174,
        7.76320939,  9.87671233,  9.94716243,  9.73581213,  8.89041096])

In [22]:
np.logspace(2, d+1, d, base=0.5)

array([0.25      , 0.125     , 0.0625    , 0.03125   , 0.015625  ,
       0.0078125 , 0.00390625, 0.00195312, 0.00097656, 0.00048828])

In [ ]:
varsortability(X, W)

In [ ]:
varsortability(X_